In [8]:
import os
import csv
import random
from datetime import datetime, timedelta

# 📂 Folder where images are stored (adjust if needed)
image_root = "images"

# ✅ Write the CSV in the current directory (ML_project/)
output_csv = "synthetic_expiry_labels.csv"

# 🏷️ Generate expiry date and class
def generate_expiry_info():
    today = datetime.today()
    days_to_expiry = random.randint(-15, 60)
    expiry_date = today + timedelta(days=days_to_expiry)

    if days_to_expiry < 0:
        label = "expired"
    elif days_to_expiry <= 15:
        label = "expiring_soon"
    else:
        label = "fresh"

    return expiry_date.strftime("%Y-%m-%d"), label

# 🖼️ Loop over images and create labels
rows = []
for category in os.listdir(image_root):
    category_path = os.path.join(image_root, category)
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            if filename.lower().endswith((".jpg", ".png", ".jpeg")):
                rel_path = os.path.join(category, filename)  # Relative path
                expiry_date, label = generate_expiry_info()
                rows.append([rel_path, expiry_date, label])

# 💾 Write to CSV
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["filename", "expiry_date", "freshness_class"])
    writer.writerows(rows)

print(f"[✔] CSV created with {len(rows)} entries at: {output_csv}")


[✔] CSV created with 4657 entries at: synthetic_expiry_labels.csv


In [9]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [10]:
df = pd.read_csv("synthetic_expiry_labels.csv")
label_map = {"expired": 0, "expiring_soon": 1, "fresh": 2}
df["label"] = df["freshness_class"].map(label_map)


In [11]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])


In [12]:
class GroceryImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.loc[idx, "filename"])
        image = Image.open(img_path).convert("RGB")
        label = self.df.loc[idx, "label"]

        if self.transform:
            image = self.transform(image)

        return image, label


In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_dataset = GroceryImageDataset(train_df, "images", transform)
test_dataset = GroceryImageDataset(test_df, "images", transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
